In [1]:
from Bio.Seq import Seq
from Bio.SeqUtils import MeltingTemp as mt
from itertools import product, islice
import re
from concurrent.futures import ProcessPoolExecutor, as_completed
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from nupack import *
import random
# Define the model
model1 = Model(material='dna', ensemble='stacking', celsius=15, sodium=1, magnesium=0.00)

In [2]:
class ComplexNotFound(Exception):
    pass

def get_complex_concentration(complexes, complex_name):
    """Retrieve the concentration of a specific complex by name."""
    for complex_key, concentration in complexes.items():
        if complex_key.name == complex_name:
            return concentration
    raise ComplexNotFound(f"Complex '{complex_name}' not found.")



In [3]:
# Assuming sequences and strands are defined as shown previously
sequences = """
TATGGCCAA
TTGGCCATA
AGGAACACA
TGTGTTCCT
TAGAAGCGA
TCGCTTCTA
ACTCAACCA
TGGTTGAGT
""".split()

strands = [Strand(seq, name=f's{i+1}') for i, seq in enumerate(sequences)]
strand_concentrations = {strand: 10e-9 for strand in strands}
tube_all = Tube(strands=strand_concentrations, complexes=SetSpec(max_size=2), name='all_in_one')

# Run complex analysis for the tube to calculate partition functions
result_all = complex_analysis(complexes=tube_all, model=model1, compute=['pfunc'])

# Calculate the equilibrium concentrations within the tube
conc_results_all = complex_concentrations(tube=tube_all, data=result_all)

# Ensure this line is correctly placed before trying to access concentrations
tube_result_all = conc_results_all.tubes[tube_all]  # Correctly define tube_result_all

# Generate complex names dynamically for every pair of strands
complex_names = [f'(s{i}+s{i+1})' for i in range(1, len(strands) + 1, 2)]

# Print out the complex names to verify
print("Generated Complex Names:", complex_names)
print()
# Assuming tube_result_all is already defined and contains the concentration results
for complex_name in complex_names:
    try:
        concentration = get_complex_concentration(tube_result_all.complex_concentrations, complex_name)
        print(f"The equilibrium concentration of {complex_name} is {concentration * 10**9:.2f} nM")
    except ComplexNotFound as e:
        print(e)



Generated Complex Names: ['(s1+s2)', '(s3+s4)', '(s5+s6)', '(s7+s8)']

The equilibrium concentration of (s1+s2) is 9.36 nM
The equilibrium concentration of (s3+s4) is 8.32 nM
The equilibrium concentration of (s5+s6) is 9.69 nM
The equilibrium concentration of (s7+s8) is 9.60 nM
